In [1]:
import pandas as pd
import numpy as np
import datetime
import math
import os
import matplotlib.pyplot as plt
from pylab import rcParams

plt.style.use('ggplot')
%pylab inline --no-import-all

#風力発電初期値
Pr=2.3 #定格電力
V_CIN=2.5 #カットイン風速
V_rat=20 #
V_CO=20 #カットアウト風速
H=30.5 #高さ
H_top=80 #風速計測の高さ（高い）
H_ref=10 #風速計測の高さ（低い）
atm_pre0=1.225 #基準大気圧
total_cost = 1575000 #コスト[yen/unit]
Op_year = 10 #稼動年数
cost = (total_cost/(Op_year*365*24)) #コスト[yen/hour/unit]

Populating the interactive namespace from numpy and matplotlib


In [2]:
pwd()

'/home/yuichiro/Miniconda/Weather/GribToCSV'

# Set or Modify parameters below!!

In [3]:
#インプットデータ
year="2019"
month="04"
day=21
########Check time value
day2=str(day+2)
day3=str(day+3)
day=str(day)
time="000000"
file_dir_path="Output_CSV/"+year+"/"+month+"/"+day
#開始日本時間（UTC+9）
##time=000000 09:00-12:00
start_TimeJP =year+'-'+month+'-'+day+' 09:00'
finish_TimeJP =year+'-'+month+'-'+day2+' 12:00'

##time=000000 21:00-00:00
#start_TimeJP =year+'-'+month+'-'+day+' 21:00'
#finish_TimeJP =year+'-'+month+'-'+day3+' 00:00'

In [4]:
###UTC time to Japan in time:000000
TimeJP = pd.date_range(start_TimeJP,finish_TimeJP,freq='H')
TimeJP.columns = ['Datetime in Japan']
Date =pd.DataFrame({'DateTime': TimeJP.values})
#Date
Date['DateTime'] = pd.to_datetime(Date['DateTime'])
#Date.dtypes
if len(Date)==52:
    print("OK")
else:
    print("Error")
    print(len(Date))
    
Date

OK


,DateTime
0,2019-04-21 09:00:00
1,2019-04-21 10:00:00
2,2019-04-21 11:00:00
3,2019-04-21 12:00:00
4,2019-04-21 13:00:00
5,2019-04-21 14:00:00
6,2019-04-21 15:00:00
7,2019-04-21 16:00:00
8,2019-04-21 17:00:00
9,2019-04-21 18:00:00


In [5]:
Date['Date']=Date['DateTime']
Date['Year']=Date['DateTime'] .dt.year
Date['Month']=Date['DateTime'] .dt.month
Date['Day']=Date['DateTime'] .dt.day
Date['Hour']=Date['DateTime'] .dt.hour

In [6]:
input_tempFH_0015 = pd.read_csv(file_dir_path+'/FH_00-15_temperature'+year+'_'+month+'_'+day+'_'+time+'.csv')
input_tempFH_1633 = pd.read_csv(file_dir_path+'/FH_16-33_temperature'+year+'_'+month+'_'+day+'_'+time+'.csv')
input_tempFH_3439 = pd.read_csv(file_dir_path+'/FH_34-39_temperature'+year+'_'+month+'_'+day+'_'+time+'.csv')
input_tempFH_4051 = pd.read_csv(file_dir_path+'/FH_40-51_temperature'+year+'_'+month+'_'+day+'_'+time+'.csv')

input_temp = pd.concat([input_tempFH_0015,input_tempFH_1633,input_tempFH_3439,\
                       input_tempFH_4051],axis=1).T.drop(["index","lons","lats"])
input_temp.columns = ['temperature']
input_temp=input_temp.reset_index()
input_temp

,index,temperature
0,temperature FH0,17.600118
1,temperature FH1,20.256368
2,temperature FH2,21.500387
3,temperature FH3,21.033712
4,temperature FH4,21.052297
5,temperature FH5,21.361105
6,temperature FH6,21.117758
7,temperature FH7,19.797048
8,temperature FH8,18.638265
9,temperature FH9,17.464651


In [7]:
input_UwindFH_0015 = pd.read_csv(file_dir_path+'/FH_00-15_Uwind'+year+'_'+month+'_'+day+'_'+time+'.csv')
input_UwindFH_1633 = pd.read_csv(file_dir_path+'/FH_16-33_Uwind'+year+'_'+month+'_'+day+'_'+time+'.csv')
input_UwindFH_3439 = pd.read_csv(file_dir_path+'/FH_34-39_Uwind'+year+'_'+month+'_'+day+'_'+time+'.csv')
input_UwindFH_4051 = pd.read_csv(file_dir_path+'/FH_40-51_Uwind'+year+'_'+month+'_'+day+'_'+time+'.csv')

input_Uwind = pd.concat([input_UwindFH_0015,input_UwindFH_1633,input_UwindFH_3439,\
                       input_UwindFH_4051],axis=1).T.drop(["index","lons","lats"]).apply(abs)
input_Uwind.columns = ['Uwind']

input_Uwind=input_Uwind.reset_index()

In [8]:
input_VwindFH_0015 = pd.read_csv(file_dir_path+'/FH_00-15_Vwind'+year+'_'+month+'_'+day+'_'+time+'.csv')
input_VwindFH_1633 = pd.read_csv(file_dir_path+'/FH_16-33_Vwind'+year+'_'+month+'_'+day+'_'+time+'.csv')
input_VwindFH_3439 = pd.read_csv(file_dir_path+'/FH_34-39_Vwind'+year+'_'+month+'_'+day+'_'+time+'.csv')
input_VwindFH_4051 = pd.read_csv(file_dir_path+'/FH_40-51_Vwind'+year+'_'+month+'_'+day+'_'+time+'.csv')

input_Vwind = pd.concat([input_VwindFH_0015,input_VwindFH_1633,input_VwindFH_3439,\
                       input_VwindFH_4051],axis=1).T.drop(["index","lons","lats"]).apply(abs)
input_Vwind.columns = ['Vwind']
input_Vwind=input_Vwind.reset_index()

In [9]:
input_radFH_0015 = pd.read_csv(file_dir_path+'/FH_00-15_radiation'+year+'_'+month+'_'+day+'_'+time+'.csv')
input_radFH_1633 = pd.read_csv(file_dir_path+'/FH_16-33_radiation'+year+'_'+month+'_'+day+'_'+time+'.csv')
input_radFH_3439 = pd.read_csv(file_dir_path+'/FH_34-39_radiation'+year+'_'+month+'_'+day+'_'+time+'.csv')
input_radFH_4051 = pd.read_csv(file_dir_path+'/FH_40-51_radiation'+year+'_'+month+'_'+day+'_'+time+'.csv')

input_rad = pd.concat([input_radFH_0015,input_radFH_1633,input_radFH_3439,\
                       input_radFH_4051],axis=1).T.drop(["index","lons","lats"])
input_rad.columns = ['radiation']
input_rad=input_rad.reset_index()
input_rad

,index,radiation
0,radiation FH0-1W/m2(avg),695.161688
1,radiation FH1-2W/m2(avg),840.941631
2,radiation FH2-3W/m2(avg),920.381489
3,radiation FH3-4W/m2(avg),927.736164
4,radiation FH4-5W/m2(avg),874.662556
5,radiation FH5-6W/m2(avg),754.650526
6,radiation FH6-7W/m2(avg),571.473579
7,radiation FH7-8W/m2(avg),384.466084
8,radiation FH8-9W/m2(avg),185.274687
9,radiation FH9-10W/m2(avg),31.562500


# Wind Calc

In [ ]:
##calc wind
wind10 = np.square(input_Uwind['Uwind'].values)+np.square(input_Vwind['Vwind'].values)
wind10 = np.sqrt(wind10)
wind10 =pd.DataFrame({'wind': wind10})
wind10.columns = ['wind_10m[m/s]']
#ind10

In [ ]:
forecast=pd.concat([Date,wind10,input_temp,input_rad,input_Uwind,input_Vwind],axis=1,sort=False)

In [ ]:
#Using Vz = Vr(Z / Zr)^(1/n) n=4

wind30_values = wind10.values*((H/10)**(1/3))
wind30 = pd.DataFrame( {'wind_30m[m/s]': wind30_values.reshape(-1,) })

In [ ]:
Pwt_values=(np.where((V_rat<=wind30_values) & (wind30_values <= V_CO), Pr,\
              np.where((V_CIN<=wind30_values) & (wind30_values <= V_rat), \
                       Pr*(wind30_values-V_CIN)/(V_rat-V_CIN),0 )))
Pwt = pd.DataFrame( {'wind output[kWh]': Pwt_values.reshape(-1,) })

In [ ]:
forecast=pd.concat([forecast,wind30,Pwt],axis=1,sort=False)
#forecast=forecast.set_index('DateTime')
forecast

# Solar calc

In [ ]:
#太陽光発電初期値

#インプットデータ
#input_rad = pd.read_csv('rad.csv')#encordingはUTF-8なので注意 unitは、MJ/m2
input_csv_parameter = pd.read_csv('PV_input_parameter.csv')

In [ ]:
#1列目以外を抽出
input_csv_parameter = input_csv_parameter.iloc[:, [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18]]

Gsc = float(input_csv_parameter.iat[0,0])#float(1.367)#the solar constant [1.367 kW/m2]
Gt_NOCT = float(input_csv_parameter.iat[0,1]) #the solar radiation at which the NOCT is defined [0.8 kW/m2]
Ta_NOCT = float(input_csv_parameter.iat[0,2])+273.15 #the ambient temperature at which the NOCT is defined [20°C]
Tc_NOCT = float(input_csv_parameter.iat[0,3])+273.15 #the nominal operating cell temperature [°C]
Tc_STC = float(input_csv_parameter.iat[0,4])+273.15 #the PV cell temperature under standard test conditions [25°C]

YPV = float(input_csv_parameter.iat[0,6]) #the rated capacity of the PV array, meaning its power output under standard test conditions [kW]
ap = float(input_csv_parameter.iat[0,7]) #Module Efficiency(Pmax)[%/℃]
beta= float(input_csv_parameter.iat[0,8])  #the slope of the surface [°]
eta_c = float(input_csv_parameter.iat[0,9]) #the solar absorptance and the solar transmittance[%]
fpv = float(input_csv_parameter.iat[0,10]) #the PV derating factor [%]
ganma = float(input_csv_parameter.iat[0,11]) #the azimuth of the surface [°]
lam=float(input_csv_parameter.iat[0,12])#the longitude [°]
phi=float(input_csv_parameter.iat[0,13])#the latitude [°]
rho_g = float(input_csv_parameter.iat[0,14]) # the ground reflectance, which is also called the albedo [%]
tau_a = float(input_csv_parameter.iat[0,15]) #the solar absorptance and the solar transmittance[%]
_Gt_STC =float(1) #the incident radiation at standard test conditions [1 kW/m2]
UL = (tau_a*Gt_NOCT)/(Tc_NOCT-Ta_NOCT)
#UL = float(input_csv_parameter.iat[0,5])
total_cost = float(input_csv_parameter.iat[0,16]) #total cost
Op_year = float(input_csv_parameter.iat[0,17]) #oparation year
cost = total_cost/(Op_year*365*24)#cost[yen/kW/h]

In [ ]:
input_csv_parameter

In [ ]:
#ωの計算
forecast['ts'] = forecast['Hour']
forecast['ω'] = (forecast['ts'].values - 12 )*15#degree
forecast['n'] = Date['DateTime'] .dt.dayofyear

In [ ]:
forecast['δ the solar declination']=23.45*np.sin(np.deg2rad(360*(284+forecast['n'].values)/365))#degree

delta=forecast['δ the solar declination'].values#degree

In [ ]:
#角度をdeg2rad
omega = np.deg2rad(forecast['ω'].values)#rad
delta =np.deg2rad(delta)#rad
phi =math.radians(phi)#rad
beta =math.radians(beta)#rad
ganma =math.radians(ganma)#rad
lam =math.radians(lam)#rad

#cosθの計算
forecast['cosθ'] = np.sin(delta)*math.sin(phi)*math.cos(beta)\
                    -np.sin(delta)*math.cos(phi)*math.sin(beta)*math.cos(ganma)\
                    +np.cos(delta)*math.cos(phi)*math.cos(beta)*np.cos(omega)\
                    +np.cos(delta)*math.sin(phi)*math.sin(beta)*math.cos(ganma)*np.cos(omega)\
                    +np.cos(delta)*math.sin(beta)*math.sin(ganma)*np.sin(omega)
                    
cos_theta = forecast['cosθ'].values

In [ ]:
#θの計算
forecast['θ(radian)'] = np.arccos(forecast['cosθ'].values)#rad

forecast['θ(degree)'] = np.rad2deg(forecast['θ(radian)'].values)#degree

Theta_deg = forecast['θ(radian)'].values#rad

#cosθzの計算
forecast['cosθz'] =math.cos(phi)*np.cos(delta)*np.cos(omega)\
                        +math.sin(phi)*np.sin(delta)

cos_theta_z = forecast['cosθz'].values

forecast['θz(radian)'] = np.arccos(forecast['cosθz'].values)#rad

forecast['θz(degree)'] = np.rad2deg(forecast['θz(radian)'].values)#degree

Theta_z_deg = forecast['θz(radian)'].values#rad

#theta degree ndarray
theta = forecast['θ(radian)'].values#rad

#theta degree ndarray
theta_z = forecast['θz(radian)'].values#rad

#decimals=1で小数点第2位を四捨五入する
cos_theta = np.round(cos_theta, decimals=1)
cos_theta_z = np.round(cos_theta_z, decimals=1)
Theta_deg =np.round(Theta_deg, decimals=0)
Theta_deg_z = np.round(Theta_z_deg, decimals=0)

cos_theta = np.cos(Theta_deg)
cos_theta_z = np.cos(Theta_z_deg)
forecast[:10]

In [ ]:
#Gon(the extraterrestrial normal radiation)[kW/m2]の計算
forecast['Gon(the extraterrestrial normal radiation)[kW/m2]'] \
=Gsc*(1+0.033*np.cos(np.deg2rad(360*forecast['n'].values/365)))#np.deg2rad

Gon =forecast['Gon(the extraterrestrial normal radiation)[kW/m2]'].values

#Go(the extraterrestrial horizontal radiation [kW/m2])の計算
forecast['Go(the extraterrestrial horizontal radiation)[kW/m2]'] \
=Gon*np.abs(cos_theta_z)

Go = forecast['Go(the extraterrestrial horizontal radiation)[kW/m2]'] .values

In [ ]:
#_Go(the extraterrestrial horizontal radiation averaged over the time step )[kW/m2]の計算

def step_minus(x):  #再帰関数　引数[2]-引数[1] ndarrayで返す ndarrayを引数
    x = list(x)
    y = [0]
    y.extend(x) #引数2
    x.extend([0]) #引数1
    y = np.array(y)
    x = np.array(x)
    ans = y - x
    return ans[1:]#[1:]or [:8760]


forecast['_Go(the extraterrestrial horizontal radiation averaged over the time step )[kW/m2]'] \
=(12/np.pi)*Gon*np.abs(np.cos(phi)*np.cos(delta)*step_minus(np.sin(omega))\
+((np.pi/180)*step_minus(omega)*np.sin(phi)*np.sin(delta)))

_Go = forecast['_Go(the extraterrestrial horizontal radiation averaged over the time step )[kW/m2]'] 

###start modified

In [ ]:
forecast['the global horizontal radiation[kW/m2]']=\
forecast['radiation'].values/1000

_G = forecast['the global horizontal radiation[kW/m2]'].values
forecast['the global horizontal radiation[kW/m2]'] = _G 

In [ ]:
#ktの計算
forecast['kt(the clearness index)'] =_G/_Go #_G2
kt = np.abs(forecast['kt(the clearness index)'].values)

_Go=kt/_G

#_Gdの計算
#２の３乗#累乗np.power(2, 3)
def if_kt(x):
    x = np.abs(x)
    ans = np.where(x>=0.8,0.165,np.where(x < 0.22, 1-0.09*x,0.95511-0.1604*x+4.388*x*x-16.638*x*x*x+12.336*x*x*x*x))
    return ans

forecast['_Gd(the diffuse radiation)[kW/m2]']=_G*if_kt(kt)
_Gd = forecast['_Gd(the diffuse radiation)[kW/m2]'].values

#_Gb(the beam radiation)の計算
forecast['_Gb(the beam radiation)[kW/m2]']=_G-_Gd

_Gb = forecast['_Gb(the beam radiation)[kW/m2]'].values

#_G
forecast['_G']=_Gd+_Gb

In [ ]:
#Rbの計算
#input_data['Rb']= cos_theta/cos_theta_z
forecast['Rb']= np.abs(forecast['cosθ'].values/forecast['cosθz'].values)

#thetaの計算がjupyterだとうまくいかない泣
#input_data['Rb']=Rb
Rb = forecast['Rb'].values

#発散の処理 要考察 yを超えるとzになる　
def div(x,y,z):
    x = abs(x)
    ans = np.where(x>y,z,x)
    return ans
#Rb = div(Rb,3,3)
#input_data['Rb']= div(Rb,3,3)
forecast['Rb']= Rb

#Afの計算
forecast['Af']=_Gb/_Go #_G2

Af = forecast['Af'].values

#fの計算
forecast['f']=np.sqrt(_Gb/_G) #_G2
    #欠損値を0で置き換える
forecast=forecast.fillna(0)
f = forecast['f'].values                         

In [ ]:
def sin3(x):
    return math.sin(x)*math.sin(x)*math.sin(x)

#_Gtの計算#２の３乗#累乗np.power(2, 3) np.power(np.sin(beta/2),3) or def 3sin(x)
_Gt=(_Gb+_Gd*Af)*Rb\
                +(_Gd*(1-Af)*((1+math.cos(beta))/2)\
                *np.abs(1+f*sin3(beta/2)))\
                +(_G*rho_g*((1-math.cos(beta))/2))
_Gt[np.isnan(_Gt)] = 0
        
forecast['_Gt']  = _Gt


In [ ]:
#Tc(the PV cell temperature [°C])の計算
forecast['temperature[K]']=forecast['temperature']+ 273.15
Ta = forecast['temperature[K]'].values


#input_data['Tc(the PV cell temperature)[°C]']\
#= Ta+_Gt*(tau_a/UL)*(1-eta_c/tau_a)

forecast['Tc(the PV cell temperature)[K]']\
= Ta+_Gt*((Tc_NOCT-Ta_NOCT)/Gt_NOCT)*(1-eta_c/tau_a)

Tc = forecast['Tc(the PV cell temperature)[K]'].values

forecast['Tc(the PV cell temperature)[°C]']=forecast['Tc(the PV cell temperature)[K]']-273.15

#Ppv(the output of the PV array)[kW]の計算   #発散の処理div(x)を使用
#input_data['Ppv(the output of the PV array)[Wh]']\
#=div(YPV*fpv*(_Gt/_Gt_STC)*np.abs((1+ap*(Tc-Tc_STC))), YPV,YPV)

#Ppv(the output of the PV array)[kW]の計算   #発散の処理div(x)を使用2
forecast['Ppv(the output of the PV array)[Wh]']\
=YPV*fpv*(_Gt/_Gt_STC)*np.abs((1+ap*(Tc-Tc_STC)))

Ppv = forecast['Ppv(the output of the PV array)[Wh]'].values

#input_data['Left'] = YPV*fpv*(_Gt/_Gt_STC)
#input_data['Right'] =np.abs((1+ap*(Tc-Tc_STC)))

forecast[:10]

In [ ]:
#入力初期値をデータフレームにする。
input_parameter_name=['Gsc','Gt_NOCT','Ta_NOCT','Tc_NOCT','Tc_STC','UL',\
                      'YPV','ap','beta','eta_c','fpv','ganma','lam','phi',\
                      'rho_g','tau_a','total_cost','Op_year','cost']

input_parameter_list=[Gsc,Gt_NOCT,Ta_NOCT,Tc_NOCT,Tc_STC,UL,\
                      YPV,ap,beta,eta_c,fpv,ganma,lam,phi,\
                      rho_g,tau_a,total_cost,Op_year,cost]

def float2DataFrame(name,list):
    df0=pd.DataFrame({str(name[0]): [list[0]]})
    for i in range(len(list)-1):
        a = float(list[i+1])
        df=pd.DataFrame({str(name[i+1]): [a]})
        df0 =pd.concat([df0, df], axis=1)
    return df0

input_parameter=float2DataFrame(input_parameter_name,input_parameter_list)
input_parameter

In [ ]:
#Timeをインデックスに設定ものをdfと定義
#df = input_data.set_index("Time") 
input_parameter.to_csv('PV_input_parameter_result.csv',encoding="SHIFT-JIS")  

In [ ]:
df = forecast.set_index('DateTime')
df.columns

In [ ]:
forecast['Rb'].describe()

# Visualizing

In [ ]:
#データの書き出し
filename = file_dir_path+'/PV_result_forecast'+year+month+day+time+'.csv'
forecast.to_csv(filename,encoding="SHIFT-JIS")  
